# Ejecutando consultas DAX sobre Power BI desde .NET 5

## Instalar la biblioteca ADOMD.NET

In [1]:
#r "nuget: Microsoft.AnalysisServices.AdomdClient.NetCore.retail.amd64, 19.16.3"

## Crear la cadena de conexión con los parámetros enviados desde Power BI

In [1]:
var pbiServer = Environment.GetEnvironmentVariable("PBI_SERVER");
var pbiDatabase = Environment.GetEnvironmentVariable("PBI_DB");

display($"Servidor: {pbiServer}");
display($"Base de datos: {pbiDatabase}");

var connectionString = $"Provider=MSOLAP;Data Source={pbiServer};Initial Catalog={pbiDatabase};";


## Ejecutar una consulta DAX que devuelve un sólo valor

In [1]:
using Microsoft.AnalysisServices.AdomdClient;

var connection = new AdomdConnection(connectionString);
connection.Open();

var command = connection.CreateCommand();

command.CommandText = "EVALUATE { [Unidades Vendidas] }";

var reader = command.ExecuteReader();

long unidadesVendidas;
if (reader.Read() && reader[0] != null)
    unidadesVendidas = reader.GetInt64(0);
    
reader.Close();

connection.Close();

display($"Total de Unidades Vendidas: {unidadesVendidas}");


## Ejecutar una consulta DAX con parámetros

In [1]:
using Microsoft.AnalysisServices.AdomdClient;

var connection = new AdomdConnection(connectionString);
connection.Open();

var command = connection.CreateCommand();

command.CommandText = @"
        EVALUATE
        {
            CALCULATE(
                [Unidades Vendidas],
                'Calendario'[Año] = @year
            )
        }";

command.Parameters.Add(new AdomdParameter("year",2020));

var reader = command.ExecuteReader();

long unidadesVendidas;
if (reader.Read() && reader[0] != null)
    unidadesVendidas = reader.GetInt64(0);
    
reader.Close();

connection.Close();

display($"Total de Unidades Vendidas: {unidadesVendidas}");

## Ejecutar una consulta DAX con ExecuteReader que devuelve una tabla

In [1]:
using Microsoft.AnalysisServices.AdomdClient;

var connection = new AdomdConnection(connectionString);
connection.Open();

var command = connection.CreateCommand();

command.CommandText = "EVALUATE 'Vendedores'";

var reader = command.ExecuteReader();

var isFirstRow = true;
while (reader.Read())
{
    if (isFirstRow)
    {
        List<string> header = new();
        for (var i = 0; i < reader.FieldCount; i++)
            header.Add(reader.GetName(i));

        display(string.Join('\t',header.ToArray()));

        isFirstRow = false;        
    }

    List<string> row = new();
    for (var i = 0; i < reader.FieldCount; i++) 
        row.Add(reader[i].ToString());

    display(string.Join('\t',row.ToArray()));
}

reader.Close();

connection.Close();

## Utilizar un DataFrame
https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/

In [1]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"

### Registrar un "Formatter" para visualizar mejor el DataFrame

In [1]:
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;

Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 40;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

### Clases auxiliares para adiconar columnas a un DataFrame

### Crear un método que ejecuta una expresión DAX y devuelve un DataFrame

In [1]:
using Microsoft.Data.Analysis;
using Microsoft.AnalysisServices.AdomdClient;
using System.Linq;

DataFrame ExecuteDax(string dax) 
{
    var connection = new AdomdConnection(connectionString);
    connection.Open();
    
    var command = connection.CreateCommand();
    
    command.CommandText = dax;
    
    var reader = command.ExecuteReader();
    
    List<StringDataFrameColumn> dfColumnList = new();
    
    var isFirstRow = true;
    while (reader.Read())
    {
        if (isFirstRow)
        {
            List<string> header = new();
            for (var i = 0; i < reader.FieldCount; i++) {
                var dfColumn = new StringDataFrameColumn(reader.GetName(i),0);
                dfColumnList.Add(dfColumn);
            }
            isFirstRow = false;        
        }
    
        List<string> row = new();
        for (var i = 0; i < reader.FieldCount; i++) 
        {
            var dfColumn = dfColumnList[i];
            dfColumn.Append(reader[i].ToString());
        }
    }
    
    reader.Close();
    
    connection.Close();
    
    return new DataFrame(dfColumnList.ToArray());
}

var df = ExecuteDax("EVALUATE 'Vendedores'");
display(df);

In [1]:
var df = ExecuteDax("EVALUATE {[Unidades Vendidas]}");
display(df);

## Más consultas DAX

### Unidades Vendidas por Categoría

In [1]:
df = ExecuteDax(
    @"EVALUATE
    SELECTCOLUMNS (
        VALUES(Productos[Categoria]),
        ""Categoria"",[Categoria],
        ""Unidades Vendidas"", [Unidades Vendidas]
    )"
);

display(df);


### Unidades Vendidas por Categoría en orden descendente

In [1]:
df = ExecuteDax (
    @"EVALUATE
    SELECTCOLUMNS (
        VALUES(Productos[Categoria]),
        ""Categoria"",[Categoria],
        ""Unidades Vendidas"", [Unidades Vendidas]
    )
    ORDER BY [Unidades Vendidas] DESC"
);

display(df);

### Unidades Vendidas por Categoría en el último año y en orden descendente

In [1]:
df = ExecuteDax (
    @"
    DEFINE
        VAR maxYear = MAX ( Calendario[Año] )
    EVALUATE
    CALCULATETABLE (
        SELECTCOLUMNS (
            VALUES ( Productos[Categoria] ),
            ""Categoria"", [Categoria],
            ""Unidades Vendidas"", [Unidades Vendidas]
        ),
        Calendario[Año] = maxYear
    )
    ORDER BY [Unidades Vendidas] DESC"
);

display(df);